In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 168.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 198.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 189.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 196.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 223.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 190.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install tqdm

In [3]:
#from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#from peft import LoraConfig
#from trl import SFTConfig, SFTTrainer
import pandas as pd
from tqdm import tqdm

In [4]:
train_df = pd.read_csv('./train.csv')
exam_df = pd.read_csv('./test.csv')

In [5]:
train_df.head()

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.0,5794.80
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.0,5591.85
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.0,5338.17
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.0,4554.42
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.0,3602.25


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(°C)         204000 non-null  float64
 4   강수량(mm)        204000 non-null  float64
 5   풍속(m/s)        204000 non-null  float64
 6   습도(%)          204000 non-null  float64
 7   일조(hr)         204000 non-null  float64
 8   일사(MJ/m2)      204000 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [41]:
system_message = """
너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
"""

question = """
건물번호: {건물번호}, 일자: {일자}, 시간: {시간}, 기온: {기온}, 강수량: {강수량}, 풍속: {풍속}, 습도: {습도}
"""

answer = """
전력소비량(kWh): {전력소비량}
"""

In [42]:
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": question.format(
                    건물번호=sample['건물번호'],
                    일자=int(sample['일시'][0:8]),  # '20250815 10' 중 '20250815' 추출 후 int 변환
                    시간=int(sample['일시'][9:11]), # '20250815 10' 중 '10' 추출 후 int 변환
                    기온=sample['기온(°C)'],
                    강수량=sample['강수량(mm)'],
                    풍속=sample['풍속(m/s)'],
                    습도=sample['습도(%)'],

                    #if isTrain:
                    #  일조=sample['일조(hr)'],
                    #  일사=sample['일사(MJ/m2)'],
                    ),
            },
            {
                "role": "assistant",
                "content": answer.format(전력소비량=sample['전력소비량(kWh)'])
            },
        ],
    }



In [43]:
print(format_data(train_df.iloc[0]))

{'messages': [{'role': 'system', 'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'}, {'role': 'user', 'content': '\n건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0\n'}, {'role': 'assistant', 'content': '\n전력소비량(kWh): 5794.8\n'}]}


In [44]:
import pandas as pd
from datetime import datetime

# 데이터프레임의 '일시' 컬럼을 datetime 객체로 변환
# format='%Y%m%d %H'는 '20250815 10'과 같은 문자열 포맷을 지정합니다.
train_df['일시_dt'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')

# 훈련 데이터와 테스트 데이터를 나눌 기준 날짜 설정
# 8월 18일 오전 0시를 기준으로 분할 (예시)
split_date = datetime(2024, 8, 16, 0, 0)

# 날짜를 기준으로 데이터 분할
# train_data에는 split_date 이전의 모든 데이터가 들어갑니다.
train_data = train_df[train_df['일시_dt'] < split_date].copy()

# test_data에는 split_date 이후의 모든 데이터가 들어갑니다.
test_data = train_df[train_df['일시_dt'] >= split_date].copy()

# 분할된 데이터 크기 확인
print(f"Train 데이터셋 크기: {len(train_data)}")
print(f"Test 데이터셋 크기: {len(test_data)}")

Train 데이터셋 크기: 182400
Test 데이터셋 크기: 21600


In [45]:
# 빈 리스트 생성
train_dataset = []

# train_data 모든 행을 순회하며 format_data 함수 적용
for index, row in tqdm(train_data.iterrows()):
  train_dataset.append(format_data(row))

# 빈 리스트 생성
test_dataset = []

# test_data 모든 행을 순회하며 format_data 함수 적용
for index, row in tqdm(test_data.iterrows()):
  test_dataset.append(format_data(row))

182400it [00:06, 28787.09it/s]
21600it [00:00, 32381.77it/s]


In [46]:
print(type(train_dataset), 'len=', len(train_dataset))
print(type(test_dataset), 'len=', len(test_dataset))


<class 'list'> len= 182400
<class 'list'> len= 21600


In [47]:
train_dataset[0]

{'messages': [{'role': 'system',
   'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'},
  {'role': 'user',
   'content': '\n건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0\n'},
  {'role': 'assistant', 'content': '\n전력소비량(kWh): 5794.8\n'}]}

In [14]:
model_id = "mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250816_merged"

In [15]:

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [48]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0
<|im_end|>
<|im_start|>assistant
<think>

</think>

전력소비량(kWh): 5794.8
<|im_end|>



In [49]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        labels = [-100] * len(input_ids)

        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    current_pos = assistant_pos + len(assistant_tokens)

                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):

                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

In [50]:
max_seq_length=4096

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 344])
어텐션 마스크 형태: torch.Size([1, 344])
레이블 형태: torch.Size([1, 344])


In [51]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 271, 127085, 16560, 130270, 126251, 72048, 11, 83556, 25715, 7, 128909, 76337, 25, 28189, 8035, 4103, 701, 130217, 7, 128909, 76337, 25, 23180, 17, 19, 701, 54116, 130000, 11, 129413, 23259, 131837, 11, 10764, 240, 235, 126299, 11, 79207, 113, 47985, 11, 83556, 92817, 131837, 11, 83556, 55054, 131837, 20401, 54116, 55902, 54248, 18411, 54116, 126641, 42039, 130270, 126251, 20401, 56419, 28754, 43590, 70582, 131837, 17877, 36055, 133085, 128555, 95617, 132612, 42905, 98005, 133032, 15235, 54070, 142713, 12802, 89659, 624, 48606, 28002, 34395, 55673, 31079, 85251, 54116, 55902, 54248, 19391, 82619, 132612, 59698, 80573, 130408, 59698, 19969, 130689, 23259, 47985, 127353, 624, 54330, 31079, 85251, 54116, 55902, 128355, 130270, 126251, 60039, 18411, 81718, 144059, 42039, 94613, 130270, 126251, 20401, 56419, 28754, 43590, 70582, 131837, 7, 43590, 23259, 5140, 239, 246, 83666, 25715, 28002, 128878, 8, 17877, 95617, 132612, 129264, 60716, 624, 57268, 125545, 9

In [52]:
test_dataset[:2]

[{'messages': [{'role': 'system',
    'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'},
   {'role': 'user',
    'content': '\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\n'},
   {'role': 'assistant', 'content': '\n전력소비량(kWh): 4920.21\n'}]},
 {'messages': [{'role': 'system',
    'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'},
   {'role

In [53]:
prompt_lst = []
label_lst = []

for prompt in tqdm(test_dataset):
    text = tokenizer.apply_chat_template(
        prompt["messages"], tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

100%|██████████| 21600/21600 [00:01<00:00, 12390.92it/s]


In [54]:
print(prompt_lst[42])

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240817, 시간: 18, 기온: 31.4, 강수량: 0.0, 풍속: 2.2, 습도: 69.0
<|im_end|>
<|im_start|>assistant


In [55]:
print(label_lst[42])


<think>

</think>

전력소비량(kWh): 7390.2
<|im_end|>



In [25]:
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    smape_val = np.mean(np.where(denominator == 0, 0, np.abs(y_true - y_pred) / denominator)) * 100
    return smape_val

In [26]:
# 모델을 평가 모드로 전환 (매우 중요)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [56]:
# ----------------- 4. 모델 예측 및 값 추출 -----------------
true_values = []
predicted_values = []
count = 0
for prompt, label in tqdm(zip(prompt_lst, label_lst), total=len(prompt_lst), desc="Evaluating model"):
    count += 1
    # 토큰화 및 모델 예측
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            pad_token_id=tokenizer.eos_token_id
        )

    # 예측된 텍스트 디코딩
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 예측된 텍스트에서 '전력소비량(kWh): ' 이후의 숫자만 추출
    try:
        # 모델이 생성한 전체 텍스트에서 'assistant' 부분만 추출
        predicted_part = predicted_text.split('assistant\n')[-1]
        predicted_power_consumption = float(predicted_part.split('전력소비량(kWh): ')[-1])

    except (ValueError, IndexError) as e:  # <-- 예외 객체를 변수 'e'로 받습니다.
        # 오류 종류와 상세 메시지를 출력
        print(f"오류 발생: {type(e).__name__}")
        print(f"오류 메시지: {e}")
        print("predicted_text=",predicted_text)
        # 숫자를 찾지 못할 경우 NaN으로 처리
        predicted_power_consumption = 0

    # 실제 값 추출
    try:
        true_power_consumption = float(label.split(': ')[-1])
    except (ValueError, IndexError):
        true_power_consumption = np.nan

    true_values.append(true_power_consumption)
    predicted_values.append(predicted_power_consumption)
    if count == 10:
        break
    


Evaluating model:   0%|          | 1/21600 [00:00<2:38:58,  2.26it/s]

오류 발생: ValueError
오류 메시지: could not convert string to float: '<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>'
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45

user

건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0

assistant
<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>


Evaluating model:   0%|          | 2/21600 [00:00<2:28:46,  2.42it/s]

오류 발생: ValueError
오류 메시지: could not convert string to float: '<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>'
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45

user

건물번호: 1, 일자: 20240816, 시간: 1, 기온: 27.4, 강수량: 0.0, 풍속: 1.5, 습도: 81.0

assistant
<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>


Evaluating model:   0%|          | 3/21600 [00:01<2:25:30,  2.47it/s]

...

오류 발생: ValueError
오류 메시지: could not convert string to float: '</tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>'
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45

user

건물번호: 1, 일자: 20240816, 시간: 6, 기온: 26.8, 강수량: 0.0, 풍속: 2.0, 습도: 80.0

assistant
</tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>


Evaluating model:   0%|          | 8/21600 [00:03<2:22:25,  2.53it/s]

오류 발생: ValueError
오류 메시지: could not convert string to float: '<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>'
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45

user

건물번호: 1, 일자: 20240816, 시간: 7, 기온: 27.0, 강수량: 0.0, 풍속: 1.2, 습도: 79.0

assistant
<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>


Evaluating model:   0%|          | 9/21600 [00:03<2:25:55,  2.47it/s]

오류 발생: ValueError
오류 메시지: could not convert string to float: '<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>\n\n<tool_call>'
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45

user

건물번호: 1, 일자: 20240816, 시간: 8, 기온: 28.4, 강수량: 0.0, 풍속: 1.9, 습도: 77.0

assistant
<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>

<tool_call>
오류 발생: ValueError
오류 메시지: could not convert string to float: ''
predicted_text= system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상

In [34]:
print(prompt_lst[0])
print(label_lst[0])

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
<|im_end|>
<|im_start|>assistant

<think>

</think>

전력소비량(kWh): 4920.21
<|im_end|>



In [29]:
# ----------------- 5. 최종 SMAPE 계산 및 출력 -----------------
# NaN 값 제거 (평가에 포함되지 않도록)
true_values = np.array(true_values)
predicted_values = np.array(predicted_values)

valid_indices = ~np.isnan(predicted_values)
y_true = true_values[valid_indices]
y_pred = predicted_values[valid_indices]

final_smape = smape(y_true, y_pred)

print(f"\nFinal SMAPE on the test dataset: {final_smape:.2f}%")


Final SMAPE on the test dataset: nan%


In [30]:
true_values

array([nan, nan, nan, ..., nan, nan, nan])

In [31]:
predicted_values

array([nan, nan, nan, ..., nan, nan, nan])